In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import re
import ipywidgets as widgets
import shutil  
from datetime import datetime
from ipywidgets import HTML, Button, VBox, Output, FloatRangeSlider
from ipywidgets import ToggleButtons
from IPython.display import display
from scipy.optimize import curve_fit
from scipy.integrate import simpson

import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import access_token
access_token.log_notebook_usage()

# === USER INPUT SECTION ===
sample_folders = sorted(glob.glob("./samples/sample*/"))  # Adjust if needed
output_base_folder = "./output_aligned/"
align_to = "first"  # or "mean"

sample_files = [sorted(glob.glob(os.path.join(folder, "*.xy"))) for folder in sample_folders]
num_files_per_sample = len(sample_files[0])
assert all(len(files) == num_files_per_sample for files in sample_files), "Mismatch in file count!"

print(f"🧪 Detected {num_files_per_sample} files per sample.")

# Core level name extraction rule
def extract_core_level_name(filename):
    base = os.path.splitext(os.path.basename(filename))[0]
    parts = base.split("_")[::-1]  # Reverse for parsing from end
    core_parts = []
    found = False

    for part in parts:
        core_parts.insert(0, part)
        if re.match(r'^[A-Za-z].*', part):  # Ends when finding first alpha-start part
            found = True
            break

    return "_".join(core_parts) if found else base

# Auto-detect core level names
core_level_names = [extract_core_level_name(os.path.basename(f)) for f in sample_files[0]]

# Display detected core levels
output_area = Output()
core_levels_display = HTML(value=f"<b>➡️ Detected core level names:</b><br>{'<br>'.join(core_level_names)}")
confirm_button = Button(description="OK", button_style='success')

def on_confirm_clicked(b):
    with output_area:
        output_area.clear_output()
        print("✅ Core levels set:", core_level_names)
        run_interface(core_level_names)

confirm_button.on_click(on_confirm_clicked)
display(VBox([core_levels_display, confirm_button, output_area]))

# === Gaussian function ===
def gaussian(x, A, mu, sigma):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2))

# === MAIN PROCESSING ===
def process_core_level(core_idx, core_name):
    spectra_raw = []
    for sample in sample_files:
        path = sample[core_idx]
        data = np.loadtxt(path)
        x, y = data[:, 0], data[:, 1]
        spectra_raw.append((x, y))

    print(f"\n🔎 Core Level: {core_name}")
    plt.figure(figsize=(8, 5))
    for x, y in spectra_raw:
        plt.plot(x, y)
    plt.title(f"Core Level: {core_name} — Raw Spectra")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.show()

    x_all = np.concatenate([x for x, _ in spectra_raw])
    x_min_slider = np.min(x_all)
    x_max_slider = np.max(x_all)

    interval_slider = FloatRangeSlider(
        value=[x_min_slider + 0.1*(x_max_slider - x_min_slider), x_max_slider - 0.1*(x_max_slider - x_min_slider)],
        min=x_min_slider,
        max=x_max_slider,
        step=0.1,
        description='Peak range:',
        continuous_update=False,
        layout={"width": "80%"}
    )

    background_range_slider = widgets.IntRangeSlider(
        value=[-20, -10],
        min=-100,
        max=0,
        step=1,
        description="Background Range (rel. to peak)",
        continuous_update=False,
        layout=widgets.Layout(width="60%")
    )

    fit_slider = FloatRangeSlider(
        value=[x_min_slider + 0.15*(x_max_slider - x_min_slider), x_max_slider - 0.15*(x_max_slider - x_min_slider)],
        min=x_min_slider,
        max=x_max_slider,
        step=0.1,
        description='Fit range:',
        continuous_update=False,
        layout={"width": "80%"}
    )

    display(background_range_slider)

    out = Output()

    # --- Add stage SelectMultiple ---
    plot_options = widgets.SelectMultiple(
        options=["R", "S", "N", "S+N", "S+N+F"],
        value=["S+N+F"],
        description="Plot(s):",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width="30%", height="120px")
    )
    run_button = Button(description="Align, Normalize & Fit", button_style='success')

    def on_click_run(b):
        with out:
            out.clear_output()
            x_min, x_max = interval_slider.value
            fit_min, fit_max = fit_slider.value
            bg_rel_min, bg_rel_max = background_range_slider.value

            peak_positions = []
            spectra_norm = []
            background_values = []
            peak_values = []
            fit_areas = []

            for x, y in spectra_raw:
                mask_peak = (x >= x_min) & (x <= x_max)
                peak_idx = np.argmax(y[mask_peak])
                peak_x = x[mask_peak][peak_idx]
                peak_y = y[mask_peak][peak_idx]
                peak_positions.append(peak_x)
                peak_values.append(peak_y)

                bg_abs_min = peak_x + bg_rel_min
                bg_abs_max = peak_x + bg_rel_max
                mask_bg = (x >= bg_abs_min) & (x <= bg_abs_max)
                bg_avg = np.mean(y[mask_bg])
                background_values.append(bg_avg)

                norm_y = (y - bg_avg) / (peak_y - bg_avg)
                spectra_norm.append((x, norm_y))

            ref_peak = peak_positions[0] if align_to == "first" else np.mean(peak_positions)

            spectra_aligned = []
            shifts_record = []
            sample_names = [os.path.basename(os.path.normpath(folder)) for folder in sample_folders]

            
            for i, ((x, y), peak_x, sample_name) in enumerate(zip(spectra_norm, peak_positions, sample_names)):
                x_shifted = x + (ref_peak - peak_x)
                spectra_aligned.append((x_shifted, y))

                shift_amount = ref_peak - peak_positions[i]
                delta = peak_values[i] - background_values[i]

                # Gaussian fit in fit range
                fit_mask = (x_shifted >= fit_min) & (x_shifted <= fit_max)
                x_fit = x_shifted[fit_mask]
                y_fit = y[fit_mask]
                try:
                    popt, _ = curve_fit(gaussian, x_fit, y_fit, p0=[1, ref_peak, 1])
                    A, mu, sigma = popt
                    y_gauss = gaussian(x_fit, *popt)
                    if x_fit[0] > x_fit[-1]:
                        x_fit = x_fit[::-1]
                        y_gauss = y_gauss[::-1]
                    area = simpson(y_gauss, x_fit)
                except Exception as e:
                    print(f"⚠️ Fit failed for sample {sample_name}: {e}")
                    area = np.nan

                fit_areas.append(area)

                shifts_record.append({
                    "sample": sample_name,
                    "core_level": core_name,
                    "file": os.path.basename(sample_files[i][core_idx]),
                    "x_shift_applied": shift_amount,
                    "peak_value": peak_values[i],
                    "background_value": background_values[i],
                    "peak_minus_background": delta,
                    "gaussian_area": area
                })

                plt.plot(x_shifted, y, label=sample_name)
                if not np.isnan(area):
                    plt.plot(x_fit, y_gauss, "--", label=f"{sample_name} fit")

                # Save aligned spectra
                output_folder = os.path.join(output_base_folder, sample_name)
                os.makedirs(output_folder, exist_ok=True)
                output_path = os.path.join(output_folder, f"{core_name}.xy")
                np.savetxt(output_path, np.column_stack((x_shifted, y)), fmt="%.6f")

            # --- Dynamic Plotting After Data Processing ---
            selected_stages = plot_options.value
            for stage in selected_stages:
                plt.figure(figsize=(8, 5))
                for i, sample_name in enumerate(sample_names):
                    x_raw, y_raw = spectra_raw[i]
                    x_norm, y_norm = spectra_norm[i]
                    x_shifted, y_shifted = spectra_aligned[i]
            
                    if stage == "R":
                        plt.plot(x_raw, y_raw, label=sample_name)
                    elif stage == "S":
                        shift = ref_peak - peak_positions[i]
                        x_s = x_raw + shift
                        plt.plot(x_s, y_raw, label=sample_name)
                    elif stage == "N":
                        plt.plot(x_raw, y_norm, label=sample_name)
                    elif stage == "S+N":
                        plt.plot(x_shifted, y_norm, label=sample_name)
                    elif stage == "S+N+F":
                        plt.plot(x_shifted, y_norm, label=sample_name)
                        if not np.isnan(fit_areas[i]):
                            x_fit = x_shifted[(x_shifted >= fit_min) & (x_shifted <= fit_max)]
                            try:
                                popt, _ = curve_fit(
                                    gaussian,
                                    x_fit,
                                    y_norm[(x_shifted >= fit_min) & (x_shifted <= fit_max)],
                                    p0=[1, ref_peak, 1]
                                )
                                y_fit = gaussian(x_fit, *popt)
                                plt.plot(x_fit, y_fit, "--", label=f"{sample_name} fit")
                            except Exception as e:
                                print(f"⚠️ Fit failed to replot: {e}")
            
                plt.title(f"{core_name} – View: {stage}")
                plt.xlabel("x")
                plt.ylabel("Intensity")
                plt.legend()
                plt.grid(True)
                plt.tight_layout()
                plt.show()



            
            #plt.title(f"{core_name} – Aligned + Normalized with Gaussian Fits")
            #plt.xlabel("x (shifted)")
            #plt.ylabel("Normalized y")
            #plt.legend()
            #plt.grid(True)
            #plt.tight_layout()
            #plt.show()

            # Bar chart of peak - background
            plt.figure(figsize=(8, 4))
            sample_labels = [r["sample"] for r in shifts_record]
            delta_values = [r["peak_minus_background"] for r in shifts_record]
            plt.bar(sample_labels, delta_values, color="skyblue")
            plt.title(f"{core_name} – Peak Minus Background per Sample")
            plt.ylabel("Peak - Background")
            plt.xlabel("Sample")
            plt.xticks(rotation=45)
            plt.grid(axis='y', linestyle="--", alpha=0.6)
            plt.tight_layout()
            plt.show()

            # Save CSV
            shifts_csv_path = os.path.join(output_base_folder, "shifts.csv")
            backup_csv_path = os.path.join(output_base_folder, "shifts_backup.csv")

            # Create DataFrame from new records and add timestamp
            df_new = pd.DataFrame(shifts_record)
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            df_new["timestamp"] = timestamp

            # === 1. Update shifts.csv (deduplicated main record) ===
            if os.path.exists(shifts_csv_path):
                df_existing = pd.read_csv(shifts_csv_path)
                mask = ~df_existing.set_index(["sample", "core_level", "file"]).index.isin(
                    df_new.set_index(["sample", "core_level", "file"]).index
                )
                df_filtered = df_existing[mask]
                df_combined = pd.concat([df_filtered, df_new.drop(columns=["timestamp"])], ignore_index=True)
            else:
                df_combined = df_new.drop(columns=["timestamp"])

            df_combined.sort_values(by=["core_level", "sample", "file"], inplace=True)
            df_combined.to_csv(shifts_csv_path, index=False)
            print(f"📄 Updated main shifts file: {shifts_csv_path}")

            # === 2. Append all new entries (with timestamp) to persistent backup ===
            if os.path.exists(backup_csv_path):
                df_backup_existing = pd.read_csv(backup_csv_path)
                df_backup_combined = pd.concat([df_backup_existing, df_new], ignore_index=True)
            else:
                df_backup_combined = df_new

            df_backup_combined.sort_values(by=["core_level", "sample", "file"], inplace=True)
            df_backup_combined.to_csv(backup_csv_path, index=False)
            print(f"🗂️  Appended results (with timestamp) to backup: {backup_csv_path}")


    run_button.on_click(on_click_run)
    display(VBox([interval_slider, fit_slider, plot_options, run_button, out]))

def run_interface(core_level_names):
    for idx, core_name in enumerate(core_level_names):
        print(f"\n==============================")
        print(f" Core Level: {core_name}")
        print(f"==============================")
        process_core_level(idx, core_name)


IndexError: list index out of range

Debug Code

In [2]:
for folder in sample_folders:
    files = glob.glob(os.path.join(folder, "*.xy"))
    print(f"{folder}: {len(files)} .xy files")

./samples/sample1/: 12 .xy files
./samples/sample2/: 12 .xy files
./samples/sample3/: 12 .xy files
./samples/sample4/: 12 .xy files
./samples/sample5/: 12 .xy files
./samples/sample6/: 12 .xy files
